# CTEs (WITH Clauses) com DuckDB

* [WITH Clause](https://duckdb.org/docs/stable/sql/query_syntax/with)

In [ ]:
import duckdb

In [ ]:
conn = duckdb.connect()

In [ ]:
conn.execute("""
    CREATE OR REPLACE TABLE vendas (
        id INTEGER,
        vendedor TEXT,
        regiao TEXT,
        valor NUMERIC,
        data DATE
    );
""")

In [ ]:
dados = [
    (1, 'Ana',    'Sul',   1000, '2024-01-01'),
    (2, 'Bruno',  'Sul',   1500, '2024-01-02'),
    (3, 'Carla',  'Norte', 1200, '2024-01-03'),
    (4, 'Ana',    'Sul',   900,  '2024-01-04'),
    (5, 'Bruno',  'Sul',   2000, '2024-01-05'),
    (6, 'Carla',  'Norte', 1100, '2024-01-06'),
    (7, 'Ana',    'Sul',   1300, '2024-01-07')
]

In [ ]:
conn.executemany(
    "INSERT INTO vendas VALUES (?, ?, ?, ?, ?);",
    dados
)

# Maior venda por região usando ROW_NUMBER

In [ ]:
display(conn.execute("""
    SELECT 
        *
    FROM (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY regiao ORDER BY valor DESC) AS posicao
        FROM vendas
    ) WHERE posicao = 1;
""").df())

In [ ]:
display(conn.execute("""
    WITH vendas_ranked AS (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY regiao ORDER BY valor DESC) AS posicao
        FROM vendas
    )
    SELECT
        *
    FROM vendas_ranked
    WHERE posicao = 1;
""").df())

# Acima da média por região

In [ ]:
display(conn.execute("""
    WITH media_por_regiao AS (
        SELECT
            regiao,
            AVG(valor) AS media_vendas
        FROM vendas
        GROUP BY regiao
    )
    SELECT
        v.vendedor,
        v.valor,
        v.regiao,
        m.media_vendas
    FROM vendas v
    JOIN media_por_regiao m ON v.regiao = m.regiao
    WHERE v.valor > m.media_vendas;
""").df())

# CTEs encadeadas: média e soma por região

In [ ]:
display(conn.execute("""
    WITH media AS (
        SELECT 
            regiao,
            AVG(valor) AS media_valor
        FROM vendas
        GROUP BY regiao
    ),
    soma AS (
        SELECT
            regiao,
            SUM(valor) AS soma_valor
        FROM vendas
        GROUP BY regiao
    )
    SELECT
        m.regiao,
        m.media_valor,
        s.soma_valor
    FROM media m
    JOIN soma s ON m.regiao = s.regiao;
""").df())

In [ ]:
conn.close()